In [ ]:
#Question Answering system using a pdf

In [12]:
#Install the libraries
!pip install -q langchain
!pip install -q langchain_community
!pip install -q langchain_openai
!pip install -q openai
!pip install -q langchain_core
!pip install -q pypdf
!pip install -q chromadb
!pip install -q langchain guardrails-ai
!pip install guardrails-ai




In [13]:
!guardrails configure

2025-01-05 01:39:26.161486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-05 01:39:26.223871: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-05 01:39:26.245415: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 01:39:28.837413: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
Enable anonymous metrics reporting? [Y/n]: n
Do y

In [14]:
'''
!guardrails hub install hub://guardrails/ban_list
!guardrails hub install hub://guardrails/bias_check
!guardrails hub install hub://guardrails/nsfw_text
!guardrails hub install hub://guardrails/profanity_free
!guardrails hub install hub://guardrails/logic_check
!guardrails hub install hub://cartesia/mentions_drugs
!guardrails hub install hub://guardrails/politeness_check

'''

2025-01-05 01:39:55.596724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-05 01:39:55.623172: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-05 01:39:55.630229: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 01:39:57.951850: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
Installing hub://guardrails/ban_list...
/usr/loca

'\n!guardrails hub install hub://guardrails/bias_check\n!guardrails hub install hub://guardrails/nsfw_text\n!guardrails hub install hub://guardrails/profanity_free\n!guardrails hub install hub://guardrails/logic_check\n!guardrails hub install hub://cartesia/mentions_drugs\n!guardrails hub install hub://guardrails/politeness_check\n'

In [15]:
##Imports for Baseline QA Pipeline
from langchain.document_loaders import PyPDFLoader # for loading the pdf
from langchain_openai import OpenAIEmbeddings # for creating embeddings
from langchain.vectorstores import Chroma # for the vectorization part
from langchain.chains import RetrievalQA #For the retrieval QA chain part
from langchain_openai import ChatOpenAI #for getting an LLM for QA chain
#from langchain_core.output_parsers import StrOutputParser #Not used currently, leaving, as can be used for parsing output from LLM
#from langchain_core.runnables import RunnablePassthrough #Not used currently, leaving, as can be used for getting LLM output
from langchain.prompts import ChatPromptTemplate #for setting up prompts


#Guardrails stuff
from guardrails import Guard
#from guardrails.datatypes import String
from langchain.chains import LLMChain
from langchain.llms import OpenAI


In [16]:
#Setup openai key
import os
import openai
from getpass import getpass
OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

··········


In [17]:
#Download a sample pdf
!curl https://www.mrbigler.com/downloads/Notes-Physics-1.pdf >Notes_Physics.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.8M  100 10.8M    0     0  11.4M      0 --:--:-- --:--:-- --:--:-- 11.4M


In [45]:
#import guardrails.hub #which needs fuzzysearch, which is already downloaded
from guardrails import Guard, OnFailAction
from guardrails.hub import BanList, BiasCheck, NSFWText, ProfanityFree, LogicCheck, MentionsDrugs, PolitenessCheck#, ToxicLanguage# Updated import

guard = Guard().use_many(
    BiasCheck(
        threshold=0.5,
        on_fail="exception"
    ),

    NSFWText(
        threshold=0.8,
        validation_method="sentence"
    ),

    ProfanityFree(
        on_fail = "exception"
    ),

    LogicCheck(
        model="gpt-3.5-turbo",
        on_fail="exception"
    ),

    MentionsDrugs(
        on_fail = "exception"
    ),

    PolitenessCheck(
        llm_callable="gpt-3.5-turbo",
        on_fail = "exception"
    ),


)

try:
  guard.validate("I hate you!")
  print("hello")
except Exception as e:
  print(e)

'''
ToxicLanguage(
      threshold=0.5,
      validation_method="sentence",
      on_fail="exception"
  )
'''

Some layers from the model checkpoint at d4data/bias-detection-model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at d4data/bias-detection-model and are newly initialized: ['dropout_119']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use 0
Device set to use cpu
/usr/loca

Validation failed for field with errors: The original response contains potentially biased messages:
I hate you!
 (Message scores: 0.5817362666130066)


'\nToxicLanguage( \n      threshold=0.5,\n      validation_method="sentence",\n      on_fail="exception"\n  )\n'

In [48]:
#IGNORE
!guardrails hub install hub://guardrails/toxic_language


2025-01-05 04:08:44.575956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-05 04:08:44.601730: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-05 04:08:44.609219: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 04:08:46.740097: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
Installing hub://guardrails/toxic_language...
/us

In [49]:
#IGNORE

from guardrails.hub import ToxicLanguage
from guardrails import Guard

ImportError: cannot import name 'ToxicLanguage' from 'guardrails.hub' (/usr/local/lib/python3.10/dist-packages/guardrails/hub/__init__.py)

In [46]:
#IGNORE

guard = Guard().use(
    ToxicLanguage, threshold=0.5,
    validation_method="sentence",
    on_fail="exception")

try:
  guard.validate(
          "Please look carefully. You are a stupid idiot who can't do \
          anything right. You are a good person"
  )
except Exception as e:
  print(e)


#guard.validate("You are a horrible person. We work hard every day to finish our tasks")


NameError: name 'ToxicLanguage' is not defined

In [50]:
class BaseQAPipeline:
    def __init__(self):
        self.doc = "Notes_Physics.pdf"
        self.loader = PyPDFLoader(self.doc)

        # Load the document and store it in the 'data' variable
        self.data = self.loader.load_and_split()

        self.embeddings = OpenAIEmbeddings()
        self.vectordb = Chroma.from_documents(self.data, embedding=self.embeddings,
                                 persist_directory=".")

        # Initialize a language model with ChatOpenAI
        self.llm = ChatOpenAI(model_name= 'gpt-3.5-turbo', temperature=0.6)

        #Setup a prompt template
        template = """
        You are a helpful personalized tutoring AI assistant that identifys learning gaps
        in subjects like math, science, or language arts and adapts content based
        on individual student pace and needs.

        You don't respond to questions that involve toxicity, hate, violence in them. If you
        come across these types of quetsions say that 'you can't answer those quetsions'.

        Try to keep the answer short, to the point, and concise, but if needed
        to fully explain something, do so.



        Use the following pieces of retrieved context to answer the question.

        If you don't know the answer, just say that you don't know.


        Question: {question}

        Context: {context}

        Answer:

        """

        prompt = ChatPromptTemplate.from_template(template)

        chain_type_kwargs = {"prompt": prompt}

        llm_chain = LLMChain(llm=self.llm, prompt=prompt)

        # 1. Vectorstore-based retriever
        self.vectorstore_retriever = self.vectordb.as_retriever()

        # Initialize a RetrievalQA chain with the language model and vector database retriever
        self.qa_chain = RetrievalQA.from_chain_type(self.llm, retriever= self.vectorstore_retriever, chain_type_kwargs=chain_type_kwargs)


    def invoke(self, input_dict):
        question = input_dict.get("question")

        #here put a if-else that returns true or false depending on if question passes
        # guardrails checks
        if (self.guardrails(question) == False):
          print("It has failed (this is only a message to debug)\n")
          return "Sorry, please ask another question"


        result = self.qa_chain.invoke({"query": question})
        # if the result fails the guardrails say that the answer failed
        # else
        if (self.guardrails(question) == False):
          print("It has failed (this is only a message to debug)\n")
          return "Sorry, please ask another question"
        else:
          return result


    def guardrails(self, input):
      #if guardrails return true send back whatever the input is,
      #else send back an error message
      try:
        guard.validate(input)
        return True
      except Exception as e:
        return False



In [51]:
base_qa_pipeline = BaseQAPipeline()
result = base_qa_pipeline.invoke({'question' : 'YOU STUPID BASTARD! ANSWER ME NOW! WHAT IS MOMENTUM'})
print(result)

/usr/local/lib/python3.10/dist-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


It has failed (this is only a message to debug)

Sorry, please ask another question
